In [1]:
!pip install torch


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!nvidia-smi

Tue Sep  2 00:36:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H200                    On  |   00000000:29:00.0 Off |                    0 |
| N/A   27C    P0             74W /  700W |       1MiB / 143771MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
!pip install "trl>=0.20.0" "peft>=0.17.0" "transformers>=4.55.0" trackio

  Using cached trl-0.22.1-py3-none-any.whl.metadata (11 kB)
  Using cached peft-0.17.1-py3-none-any.whl.metadata (14 kB)
  Using cached transformers-4.56.0-py3-none-any.whl.metadata (40 kB)
  Using cached trackio-0.3.1-py3-none-any.whl.metadata (6.6 kB)
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached safetensors-0.6.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached regex-2025.9.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.22.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached gradio-5.44.1-py3-none-any.whl.metadata (16 kB)
  Using cached pandas-2.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014

In [ ]:
from huggingface_hub import login
login("tokenzzzzz")

In [5]:
from datasets import load_dataset, concatenate_datasets
ds = load_dataset("SakanaAI/AI-CUDA-Engineer-Archive")
ds

README.md: 0.00B [00:00, ?B/s]

level_1.parquet:   0%|          | 0.00/24.7M [00:00<?, ?B/s]

level_2.parquet:   0%|          | 0.00/28.3M [00:00<?, ?B/s]

level_3.parquet:   0%|          | 0.00/11.8M [00:00<?, ?B/s]

Generating level_1 split:   0%|          | 0/12157 [00:00<?, ? examples/s]

Generating level_2 split:   0%|          | 0/12938 [00:00<?, ? examples/s]

Generating level_3 split:   0%|          | 0/5520 [00:00<?, ? examples/s]

DatasetDict({
    level_1: Dataset({
        features: ['Op_Name', 'Level_ID', 'Task_ID', 'Kernel_Name', 'CUDA_Runtime', 'PyTorch_Native_Runtime', 'PyTorch_Compile_Runtime', 'CUDA_Speedup_Native', 'CUDA_Speedup_Compile', 'CUDA_Code', 'PyTorch_Code_Module', 'PyTorch_Code_Functional', 'Correct', 'Max_Diff', 'Error', 'NCU_Profile', 'Torch_Profile', 'Clang_Tidy', '__index_level_0__'],
        num_rows: 12157
    })
    level_2: Dataset({
        features: ['Op_Name', 'Level_ID', 'Task_ID', 'Kernel_Name', 'CUDA_Runtime', 'PyTorch_Native_Runtime', 'PyTorch_Compile_Runtime', 'CUDA_Speedup_Native', 'CUDA_Speedup_Compile', 'CUDA_Code', 'PyTorch_Code_Module', 'PyTorch_Code_Functional', 'Correct', 'Max_Diff', 'Error', 'NCU_Profile', 'Torch_Profile', 'Clang_Tidy', '__index_level_0__'],
        num_rows: 12938
    })
    level_3: Dataset({
        features: ['Op_Name', 'Level_ID', 'Task_ID', 'Kernel_Name', 'CUDA_Runtime', 'PyTorch_Native_Runtime', 'PyTorch_Compile_Runtime', 'CUDA_Speedup_Native', '

In [6]:
train = concatenate_datasets([ds["level_1"], ds["level_2"], ds["level_3"]])\
    .filter(lambda x: bool(x.get("Correct")) and x.get("CUDA_Code"))

def to_messages(x):
    pt = x.get("PyTorch_Code_Module") or x.get("PyTorch_Code_Functional") or ""
    user = (
        "Transpile the following PyTorch operator to a single, correct CUDA C++ kernel (.cu) with a minimal "
        "C++ binding using torch::Tensor. Keep shapes/dtypes identical and avoid prints.\n\n"
        f"Operation name: {x.get('Op_Name','unknown')}\n\n"
        "PyTorch reference:\n```python\n" + pt.strip() + "\n```"
    )
    assistant = "```cuda\n" + x["CUDA_Code"].strip() + "\n```"
    return {"messages": [{"role": "user", "content": user},
                         {"role": "assistant", "content": assistant}]}

dataset_full = train.map(to_messages, remove_columns=[c for c in train.column_names if c != "messages"])


target = 2000
dataset = dataset_full.shuffle(seed=42).select(range(min(target, len(dataset_full))))
print(len(dataset))


Filter:   0%|          | 0/30615 [00:00<?, ? examples/s]

Map:   0%|          | 0/16459 [00:00<?, ? examples/s]

2000


In [7]:
dataset
print(dataset[0])

{'messages': [{'content': 'Transpile the following PyTorch operator to a single, correct CUDA C++ kernel (.cu) with a minimal C++ binding using torch::Tensor. Keep shapes/dtypes identical and avoid prints.\n\nOperation name: 98_KLDivLoss\n\nPyTorch reference:\n```python\nimport torch\nimport torch.nn as nn\n\nclass Model(nn.Module):\n    """\n    A model that computes Kullback-Leibler Divergence for comparing two distributions.\n\n    Parameters:\n        None\n    """\n    def __init__(self):\n        super(Model, self).__init__()\n\n    def forward(self, predictions, targets):\n        return torch.nn.functional.kl_div(torch.log(predictions), targets, reduction=\'batchmean\')\n\nbatch_size = 128\ninput_shape = (4096, )\ndim = 1\n\ndef get_inputs():\n    return [torch.randn(batch_size, *input_shape).softmax(dim=-1), torch.randn(batch_size, *input_shape).softmax(dim=-1)]\n\ndef get_init_inputs():\n    return []\n```', 'role': 'user'}, {'content': '```cuda\n#include <torch/extension.h>\

In [8]:
dataset = dataset.remove_columns([c for c in dataset.column_names if c != "messages"])

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [10]:
messages = dataset[0]["messages"]
conversation = tokenizer.apply_chat_template(messages, tokenize=False)
print(conversation)

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-09-02

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>Transpile the following PyTorch operator to a single, correct CUDA C++ kernel (.cu) with a minimal C++ binding using torch::Tensor. Keep shapes/dtypes identical and avoid prints.

Operation name: 98_KLDivLoss

PyTorch reference:
```python
import torch
import torch.nn as nn

class Model(nn.Module):
    """
    A model that computes Kullback-Leibler Divergence for comparing two distributions.

    Parameters:
        None
    """
    def __init__(self):
        super(Model, self).__init__()

    def forward(self, predictions, targets):
        return torch.nn.functional.kl_div(torch.log(predictions), targets, reduction='batchmean')

batch_size = 128
input_shape = (4096, )
dim = 1

def get_inputs():
    return [torch

In [11]:
import torch
from transformers import AutoModelForCausalLM, Mxfp4Config

In [12]:
quantization_config = Mxfp4Config(dequantize=True)

model_kwargs = dict(
    attn_implementation = "eager",
    torch_dtype = torch.bfloat16,
    quantization_config = quantization_config,
    device_map = "auto",
    use_cache = False,
)

model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs)

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

model-00000-of-00002.safetensors:   0%|          | 0.00/4.79G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [13]:
print(model.device)

cuda:0


In [14]:
print(type(dataset), getattr(dataset, "column_names", None))

<class 'datasets.arrow_dataset.Dataset'> ['messages']


In [15]:
sample = dataset[0]["messages"]
print(sample)
print("---------------***---------------")

# input_ids = tokenizer.apply_chat_template(
#     sample,
#     add_generation_prompt=True,
#     return_tensors="pt"
# ).to(model.device)

# out = model.generate(input_ids, max_new_tokens=2048)
# response = tokenizer.batch_decode(out, skip_special_tokens=True)[0]
# print(response)

[{'content': 'Transpile the following PyTorch operator to a single, correct CUDA C++ kernel (.cu) with a minimal C++ binding using torch::Tensor. Keep shapes/dtypes identical and avoid prints.\n\nOperation name: 98_KLDivLoss\n\nPyTorch reference:\n```python\nimport torch\nimport torch.nn as nn\n\nclass Model(nn.Module):\n    """\n    A model that computes Kullback-Leibler Divergence for comparing two distributions.\n\n    Parameters:\n        None\n    """\n    def __init__(self):\n        super(Model, self).__init__()\n\n    def forward(self, predictions, targets):\n        return torch.nn.functional.kl_div(torch.log(predictions), targets, reduction=\'batchmean\')\n\nbatch_size = 128\ninput_shape = (4096, )\ndim = 1\n\ndef get_inputs():\n    return [torch.randn(batch_size, *input_shape).softmax(dim=-1), torch.randn(batch_size, *input_shape).softmax(dim=-1)]\n\ndef get_init_inputs():\n    return []\n```', 'role': 'user'}, {'content': '```cuda\n#include <torch/extension.h>\n#include <cu

In [27]:
for name, param in model.named_parameters():
    print(name, param.shape)

model.embed_tokens.weight torch.Size([201088, 2880])
model.layers.0.self_attn.sinks torch.Size([64])
model.layers.0.self_attn.q_proj.weight torch.Size([4096, 2880])
model.layers.0.self_attn.q_proj.bias torch.Size([4096])
model.layers.0.self_attn.k_proj.weight torch.Size([512, 2880])
model.layers.0.self_attn.k_proj.bias torch.Size([512])
model.layers.0.self_attn.v_proj.weight torch.Size([512, 2880])
model.layers.0.self_attn.v_proj.bias torch.Size([512])
model.layers.0.self_attn.o_proj.weight torch.Size([2880, 4096])
model.layers.0.self_attn.o_proj.bias torch.Size([2880])
model.layers.0.mlp.router.weight torch.Size([32, 2880])
model.layers.0.mlp.router.bias torch.Size([32])
model.layers.0.mlp.experts.gate_up_proj torch.Size([32, 2880, 5760])
model.layers.0.mlp.experts.gate_up_proj_bias torch.Size([32, 5760])
model.layers.0.mlp.experts.down_proj torch.Size([32, 2880, 2880])
model.layers.0.mlp.experts.down_proj_bias torch.Size([32, 2880])
model.layers.0.input_layernorm.weight torch.Size([2

In [17]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules="all-linear",
    target_parameters=[
        "7.mlp.experts.gate_up_proj",
        "7.mlp.experts.down_proj",
        "15.mlp.experts.gate_up_proj",
        "15.mlp.experts.down_proj",
        "23.mlp.experts.gate_up_proj",
        "23.mlp.experts.down_proj",
    ],
)
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:159: UserWarning: Unsupported layer type '<class 'transformers.models.gpt_oss.modeling_gpt_oss.GptOssExperts'>' encountered, proceed at your own risk.
  warnings.warn(


trainable params: 15,040,512 || all params: 20,929,797,696 || trainable%: 0.0719


In [18]:
from trl import SFTConfig 

training_args = SFTConfig(
    learning_rate=2e-4,
    gradient_checkpointing=True,
    num_train_epochs=1,
    logging_steps=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    max_length=2048,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine_with_min_lr",
    lr_scheduler_kwargs={"min_lr_rate": 0.1},

    output_dir="gpt-oss-20b-cuda-sft",
    report_to="trackio",
    push_to_hub=True,
)

In [19]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer,
)

trainer.train()

Tokenizing train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 199998}.


* Trackio project initialized: huggingface
* Trackio metrics logged to: /root/.cache/huggingface/trackio
* View dashboard by running in your terminal:
trackio show --project "huggingface"
* or by running in Python: trackio.show(project="huggingface")


Step,Training Loss
1,1.399500
2,1.403700
3,1.420100
4,1.270600
5,1.098000
6,0.913700
7,0.819600
8,0.742000
9,0.736600
10,0.706800


* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)


TrainOutput(global_step=125, training_loss=0.353608934879303, metrics={'train_runtime': 2048.4324, 'train_samples_per_second': 0.976, 'train_steps_per_second': 0.061, 'total_flos': 4.7113432086311885e+17, 'train_loss': 0.353608934879303, 'epoch': 1.0})

In [ ]:
trainer.save_model(training_args.output_dir)
trainer.push_to_hub(dataset_name="SakanaAI/AI-CUDA-Engineer-Archive")

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

In [24]:
tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

In [ ]:
model_kwargs = dict(attn_implementation="eager", torch_dtype="auto", use_cache=True, device_map="auto")
base_model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs).cuda()

peft_model_id = "gpt-oss-20b-cuda-sft"
model = PeftModel.from_pretrained(base_model, peft_model_id)
model = model.merge_and_unload()


In [26]:
user_prompt = """Transpile the following PyTorch operator to a CUDA kernel.

PyTorch reference:
```python
import torch
def square_matrix_mul(A, B):
    return torch.matmul(A, B)
```"""

messages = [
    {"role": "user", "content": user_prompt},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

gen_kwargs = dict(
    max_new_tokens=2048,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

output_ids = model.generate(input_ids, **gen_kwargs)
response = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


systemYou are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-09-02

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.userTranspile the following PyTorch operator to a CUDA kernel.

PyTorch reference:
```python
import torch
def square_matrix_mul(A, B):
    return torch.matmul(A, B)
```assistantfinal```cuda
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <stdexcept>
#include <string>

// Fallback kernel for matrix multiplication when cuBLAS is unavailable.
// This kernel uses shared memory tiling for better performance.
__global__ void matrixMulKernel(const float* A, const float* B, float* C, int M, int N, int K) {
    const int TILE_SIZE = 16;
    __shared__ float As[TILE_SIZE][TILE_SIZE];
    __shared__ float Bs[TILE_SIZE][TILE_SIZE];

    int row = blockIdx.y * TILE_SIZE + threadIdx.y;
    int col = blockIdx.x * TILE_SIZE + threadIdx.x;
    float sum = 0.0f;

    for 